In [2]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
! pip install datasets transformers rouge-score jieba

In [ ]:
! pip install --upgrade ipywidgets tqdm
! jupyter nbextension enable --py widgetsnbextension

In [ ]:
! pip install lawrouge

In [12]:
# 压缩下载训练完成的模型
import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()


if __name__ == "__main__":
    # 文件夹路径
    packagePath = '/kaggle/working/results/best'#需要压缩的文件路径
    zipPath = '/kaggle/working/output.zip'#保存压缩包的路径
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")
    print(datetime.datetime.utcnow())


打包完成
2024-07-25 17:54:43.378698


In [ ]:
# 训练模型
import torch
import datasets
import lawrouge
import numpy as np

from typing import List, Dict
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer,
                          BartForConditionalGeneration)
batch_size = 16
epochs = 5
# 最大输入长度
max_input_length = 512
# 最大输出长度
max_target_length = 128
learning_rate = 1e-04
# 读取数据

dataset = load_dataset('json', data_files='/kaggle/input/nlpcc2017-data0/nlpcc2017_clean.json', field='data')
# 加载tokenizer,中文bart使用bert的tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/bart-base-chinese/transformers/default/1/bart-base-chinese")
# 调整数据格式
def flatten(example):
    return {
        "document": example["content"],
        "summary": example["title"],
        "id": "0"
    }

# 将原始数据中的content和title转换为document和summary
dataset = dataset["train"].map(flatten, remove_columns=["title", "content"])
print(dataset)
# 划分数据集
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.1,shuffle=True,seed=42).values()
train_dataset, test_dataset = train_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42).values()
datasets = datasets.DatasetDict({"train":train_dataset,"validation": valid_dataset,"test":test_dataset})
print(datasets["train"][2])
print(datasets["validation"][2])
print(datasets["test"][2])
print("数据转换完毕")
def preprocess_function(examples):
    """
    document作为输入，summary作为标签
    """
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = datasets
tokenized_datasets = tokenized_datasets.map(preprocess_function, batched=True, remove_columns=["document", "summary", "id"])
# print(tokenized_datasets["train"][2].keys())
# print(tokenized_datasets["train"][2])

def collate_fn(features: Dict):
    batch_input_ids = [torch.LongTensor(feature["input_ids"]) for feature in features]
    batch_attention_mask = [torch.LongTensor(feature["attention_mask"]) for feature in features]
    batch_labels = [torch.LongTensor(feature["labels"]) for feature in features]

    # padding
    batch_input_ids = pad_sequence(batch_input_ids, batch_first=True, padding_value=0)
    batch_attention_mask = pad_sequence(batch_attention_mask, batch_first=True, padding_value=0)
    batch_labels = pad_sequence(batch_labels, batch_first=True, padding_value=-100)
    return {
        "input_ids": batch_input_ids,
        "attention_mask": batch_attention_mask,
        "labels": batch_labels
    }

# 构建DataLoader来验证collate_fn
dataloader = DataLoader(tokenized_datasets["test"], shuffle=False, batch_size=4, collate_fn=collate_fn)
batch = next(iter(dataloader))
# print(batch)

print("开始模型训练")

model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/bart-base-chinese/transformers/default/1/bart-base-chinese")
# output = model(**batch) # 验证前向传播
# print(output)
print("加载预训练模型")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # 将id解码为文字
    predictions = np.where(predictions != -100,predictions,tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # 替换标签中的-100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # 去掉解码后的空格
    decoded_preds = ["".join(pred.replace(" ", "")) for pred in decoded_preds]
    decoded_labels = ["".join(label.replace(" ", "")) for label in decoded_labels]
    # 分词计算rouge
    # decoded_preds = [" ".join(jieba.cut(pred.replace(" ", ""))) for pred in decoded_preds]
    # decoded_labels = [" ".join(jieba.cut(label.replace(" ", ""))) for label in decoded_labels]
    # 计算rouge
    rouge = lawrouge.Rouge()
    result = rouge.get_scores(decoded_preds, decoded_labels,avg=True)
    result = {'rouge-1': result['rouge-1']['f'], 'rouge-2': result['rouge-2']['f'], 'rouge-l': result['rouge-l']['f']}
    result = {key: value * 100 for key, value in result.items()}
    return result
early_stopping=False
# 设置训练参数
args = Seq2SeqTrainingArguments(
    output_dir="results", # 模型保存路径
    num_train_epochs=epochs,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    warmup_steps=500,
    weight_decay=0.001,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=500,
    eval_strategy="steps",
    save_total_limit=3,
    generation_max_length=128, # 生成的最大长度
    generation_num_beams=1, # beam search
    load_best_model_at_end=True,
    metric_for_best_model="rouge-1"
)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=collate_fn,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
train_result = trainer.train()
# 打印验证集上的结果
print(trainer.evaluate(tokenized_datasets["validation"]))
# 打印测试集上的结果
print(trainer.evaluate(tokenized_datasets["test"]))
# 保存最优模型
trainer.save_model("results/best")


# 加载训练好的模型
model = BartForConditionalGeneration.from_pretrained("results/best")
# model = model.to("cuda")
# 从测试集中挑选4个样本
test_examples = test_dataset["document"][:4]
print(test_examples)
inputs = tokenizer(
        test_examples,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
        return_tensors="pt",
    )
input_ids = inputs.input_ids.to(model.device)
attention_mask = inputs.attention_mask.to(model.device)
# 生成
outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=128)
# 将token转换为文字
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str = [s.replace(" ","") for s in output_str]
print(output_str)

In [13]:
# 加载训练好的模型  ----验证模型
model = BartForConditionalGeneration.from_pretrained("results/best")
# model = model.to("cuda")
# 从测试集中挑选4个样本
test_examples = test_dataset["document"][:4]
print(test_examples)
inputs = tokenizer(
        test_examples,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
        return_tensors="pt",
    )
input_ids = inputs.input_ids.to(model.device)
attention_mask = inputs.attention_mask.to(model.device)
# 生成
outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=128)
# 将token转换为文字
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str = [s.replace(" ","") for s in output_str]
print(output_str)

['22大连市气象台2015年03月16日03时50分发布大雾橙色预警信号:预计16日凌晨到上午,大连市区、金州区及周边海域的局部地区将出现能见度小于200米的浓雾天气,请注意防范。图例标准防御指南6小时内可能出现能见度小于200米的浓雾,或者已经出现能见度小于200米、大于等于50米的浓雾且可能持续。1、有关部门和单位按照职责做好防雾工作;2、机场、高速公路、轮渡码头等单位加强调度指挥;3、驾驶人员必须严格控制车、船的行进速度;4、减少户外活动。', '12月24日,据成都全搜索报道,12月24日下午,成都市召开领导干部大会。省委常委、省委组织部部长范锐平宣布中央和四川省委关于唐良智同志任职的决定,唐良智任市委委员、常委、副书记,提名为成都市人民政府市长人选。范锐平代表省委书记王东明对成都工作提出四点要求。省委常委、市委书记黄新初主持会议并作重要讲话。唐良智1979年9月至1983年8月,华中工学院固体电子系半导体物理与器件专业学习;1983年8月至1985年4月,邮电部上海第一研究所工程技术人员;1985年4月至1987年1月,邮电部武汉邮电科学研究院工程技术人员;1987年1月至1991年2月,武汉市东湖新技术开发区管理办公室主任科员(1990年2月至1990年4月,赴日本JICA进修);1991年2月至1995年9月,东湖新技术开发区新技术创业中心副主任、党委书记;1995年9月至1998年7月,东湖新技术开发区管委会政策法规处副处长、处长(1992年9月至1995年12月,在华中理工大学西方经济学专业在职硕士研究生学习,获经济学硕士学位;1998年1月至1998年7月,兼任东湖高新技术股份公司总经理);1998年7月至2000年2月,东湖新技术开发区管委会副主任、党委委员;2000年2月至2001年2月,东湖新技术开发区管委会副主任、党委副书记(2000年10月明确正局级);2001年2月至2001年10月,市委武汉东湖新技术开发区工委副书记、管委会副主任;2001年10月至2003月4月,硚口区委副书记,区政府代理区长、区长(2002年1月任)(1999年9月至2002年6月,在华中科技大学西方经济学专业在职博士研究生学习,获经济学博士学位);2003年4月至2005年5月,武汉东湖新技术开发区管委会主任、工委副书记;2005年5月至2005年10月,武